**Handwritten Character Recognition Model**

In [86]:
#import library
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from tensorflow.keras import layers, models
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten

import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [87]:
#Load Dataset
emnist = tfds.load('emnist/letters', split='train', as_supervised=True)
mnist = tfds.load('mnist', split='train', as_supervised=True)

*Preprocessing and preparing Dataset*

In [88]:
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.expand_dims(image, -1)
    return image, label

def shift_emnist(image, label):
    return image, label + 9  # EMNIST letters: 1-26 → 10-35

In [89]:
mnist = mnist.map(normalize)
emnist = emnist.map(shift_emnist).map(normalize)

In [90]:
#Combine Two dataset
combined = mnist.concatenate(emnist)
combined = combined.shuffle(120_000, seed=42)

In [91]:
#split dataset
total_samples = 70000 + 88800 // 2  # ~112800
train_size = int(0.8 * total_samples)

train_ds = combined.take(train_size)
test_ds = combined.skip(train_size)

In [92]:
BATCH_SIZE = 64
train_ds = train_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [93]:
for images, labels in train_ds.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch:", labels.numpy())

Image batch shape: (64, 28, 28, 1, 1)
Label batch: [28 12 11 12 24  6  9 29  8 10  8 28 14  4 18  9 32  7  6  9  6 27 24  2
 27  3 32 10 21  9  6  0  7 11 30 23 11 12 20 18  3  2 14 23  4  6 25  5
 14 13  0  6 33  3 32 27 19 26  3  1 12 28  2  4]


*Buiding Model*

In [103]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(36,activation='softmax'))

In [104]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [105]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_17 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 1, 1, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 36)             │         2,340 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 103,268 (403.39 KB)

 Trainable params: 103,268 (403.39 KB)

 Non-trainable params: 0 (0.00 B)

*Model training*

In [106]:
history = model.fit( train_ds,epochs=20,validation_data=test_ds)

Epoch 1/10
1430/1430 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.6142 - loss: 1.3757 - val_accuracy: 0.8517 - val_loss: 0.4599
Epoch 2/10
1430/1430 ━━━━━━━━━━━━━━━━━━━━ 22s 11ms/step - accuracy: 0.8746 - loss: 0.3907 - val_accuracy: 0.8957 - val_loss: 0.3138
Epoch 3/10
1430/1430 ━━━━━━━━━━━━━━━━━━━━ 38s 9ms/step - accuracy: 0.9059 - loss: 0.2894 - val_accuracy: 0.9048 - val_loss: 0.2838
Epoch 4/10
1430/1430 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9178 - loss: 0.2489 - val_accuracy: 0.9246 - val_loss: 0.2268
Epoch 5/10
1430/1430 ━━━━━━━━━━━━━━━━━━━━ 21s 11ms/step - accuracy: 0.9281 - loss: 0.2160 - val_accuracy: 0.9297 - val_loss: 0.2063
Epoch 6/10
1430/1430 ━━━━━━━━━━━━━━━━━━━━ 20s 11ms/step - accuracy: 0.9318 - loss: 0.2021 - val_accuracy: 0.9402 - val_loss: 0.1765
Epoch 7/10
1430/1430 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.9382 - loss: 0.1813 - val_accuracy: 0.9383 - val_loss: 0.1758
Epoch 8/10
1430/1430 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9422 - l

In [109]:
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")

Final Validation Accuracy: 0.9512


*Model Save*

In [110]:
model.save("mnist_emnist_combined_cnn.h5")